Code for the GUI for Project 1

In [2]:
import tkinter as tk 
from tkinter import messagebox, ttk
import tkinter.scrolledtext as scrolledtext
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from neo4j import GraphDatabase

In [3]:
uri = "mongodb://localhost:27017"
client = MongoClient(uri, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
db = client['Project1']
nodes = db['nodes']
edges = db['edges']

Pinged your deployment. You successfully connected to MongoDB!


In [4]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"
auth=(username, password)
driver = GraphDatabase.driver(uri, auth=(username, password))

In [5]:
def graphq1(driver, node_id):
    query = """
        MATCH (n {id: $node_id})
        OPTIONAL MATCH (n)-[r1]->(target)
        WHERE r1.metaedge IN ['DlA', 'DaG']
        OPTIONAL MATCH (source)-[r2]->(n)
        WHERE r2.metaedge IN ['CpD', 'CtD']
        RETURN n, COLLECT(DISTINCT target) AS target_nodes, COLLECT(DISTINCT source) AS source_nodes
    """
    with driver.session() as session:
        result = session.run(query, node_id=node_id)
        result = result.data()  

    names = []

    disease_name = result[0]['n']['name']
    names.append('Disease Name')
    names.append(disease_name)

    names.append('Anatomy and Genes')
    for node in result[0]['target_nodes']:
        names.append(node['name'])

    names.append('Compounds')
    for node in result[0]['source_nodes']:
        names.append(node['name'])

    return names


In [6]:
def graphq2(driver):
    query = """
    MATCH (disease)
    WHERE disease.kind = 'Disease'
    AND NOT EXISTS {
        MATCH (disease)-[]-(compound)
        WHERE compound.kind = 'Compound'
    }
    WITH COLLECT(disease) AS diseases
    MATCH (disease)-[disAnatRel]->(anatomy)
    WHERE disease IN diseases
      AND disAnatRel.metaedge = 'DlA' 
      AND anatomy.kind = 'Anatomy'
    WITH COLLECT(DISTINCT anatomy) AS anatomies
    MATCH (compound)-[compoundGeneRel]->(gene)<-[anatomyGeneRel]-(anatomy)
    WHERE compound.kind = 'Compound'
      AND gene.kind = 'Gene'
      AND anatomy IN anatomies
      AND (
          (compoundGeneRel.metaedge = 'CuG' AND anatomyGeneRel.metaedge = 'AdG')
          OR
          (compoundGeneRel.metaedge = 'CdG' AND anatomyGeneRel.metaedge = 'AuG')
      )
    
    RETURN DISTINCT compound
    """
    
    with driver.session() as session:
        result = session.run(query)
        result = list(result.data())

    compound_names = [item['compound']['name'] for item in result]
    compound_names

    return compound_names

In [7]:
def mongoq1(node_id):
    result = nodes.aggregate([{
        "$match": {
            "$or": [

                # Include original node to get name
                {"id": node_id}, 

                # Disease is associated with genes or located in anatomy
                {"id": {"$in": list(edges.distinct("target", {
                    "source": node_id,
                    "metaedge": {"$in": ["DlA", "DaG"]}}))}
                },

                # Compound treats or palliates disease 
                {"id": {"$in": list(edges.distinct("source", {
                    "target": node_id,
                    "metaedge": {"$in": ["CpD", "CtD"]}}))}
                }
            ]
        }
    }])
    genes = []
    diseases = []
    compounds = []
    anatomy = []

    for node in result:
        kind = node["kind"]
        name = node["name"]

        if kind == "Gene":
            genes.append(name)
        elif kind == "Disease":
            diseases.append(name)
        elif kind == "Compound":
            compounds.append(name)
        elif kind == "Anatomy":
            anatomy.append(name)

    output = ['Disease Name'] + diseases + ['Gene Names'] + genes + ['Compound Names'] + compounds + ['Anatomy Names'] + anatomy
    return output
    


In [8]:
def mongoq2():
    pipeline = [
        # Start with compounds only
        {"$match": {"kind": "Compound"}},
        
        # Lookup compound->gene edges with projection to reduce data transfer
        {"$lookup": {
            "from": "edges",
            "let": {"compound_id": "$id"},
            "pipeline": [
                {"$match": {
                    "$expr": {
                        "$and": [
                            {"$eq": ["$source", "$$compound_id"]},
                            {"$in": ["$metaedge", ["CdG", "CuG"]]}
                        ]
                    }
                }},
                # Only keep the target (gene_id) and metaedge fields
                {"$project": {
                    "target": 1,
                    "metaedge": 1,
                    "_id": 0
                }}
            ],
            "as": "compound_gene_edges"
        }},
        
        # Filter out compounds without gene connections
        {"$match": {"compound_gene_edges": {"$ne": []}}},
        
        # Unwind to process each gene connection individually
        {"$unwind": "$compound_gene_edges"},
        
        # Lookup anatomy->gene connections for the specific gene
        {"$lookup": {
            "from": "edges",
            "let": {"gene_id": "$compound_gene_edges.target", 
                   "compound_metaedge": "$compound_gene_edges.metaedge"},
            "pipeline": [
                {"$match": {
                    "$expr": {
                        "$and": [
                            {"$eq": ["$target", "$$gene_id"]},
                            {"$in": ["$metaedge", ["AuG", "AdG"]]},
                            # Match relationship patterns directly
                            {"$or": [
                                {"$and": [
                                    {"$eq": ["$$compound_metaedge", "CuG"]},
                                    {"$eq": ["$metaedge", "AdG"]}
                                ]},
                                {"$and": [
                                    {"$eq": ["$$compound_metaedge", "CdG"]},
                                    {"$eq": ["$metaedge", "AuG"]}
                                ]}
                            ]}
                        ]
                    }
                }},
                {"$limit": 1} # Still keep the limit 1 for efficiency
            ],
            "as": "anatomy_gene_edges"
        }},
        
        # Keep only compounds that have matching anatomy-gene connections
        {"$match": {"anatomy_gene_edges": {"$ne": []}}},
        
        # Group by compound ID to remove duplicates, keep just the name
        {"$group": {
            "_id": "$id",
            "name": {"$first": "$name"}
        }},
        
        # Final projection
        {"$project": {
            "_id": 0,
            "name": 1
        }}
    ]
    
    result = list(nodes.aggregate(pipeline))
    names = [entry['name'] for entry in result]
    return names

When running mongo query2, query takes about a minute, interacting with the GUI can break it

In [9]:
def on_request_button_click():
    db_selection = db_var.get()
    query_selection = query_var.get()
    
    # Check if disease ID is needed and provided
    if query_selection == "Query 1":
        disease_id = disease_entry.get()
        if not disease_id.strip():
            messagebox.showwarning("Input Required", "Please enter a disease ID for Query 1")
            return
    
    # Disable the execute button during query execution
    execute_button.config(state=tk.DISABLED)
    status_bar.config(text="Executing query...")

    # Clear previous results
    results_text.config(state=tk.NORMAL)
    results_text.delete(1.0, tk.END)
    results_text.insert(tk.END, "Executing query, please wait...")
    results_text.config(state=tk.DISABLED)
    
    try:
        if db_selection == "Graph":
            if query_selection == "Query 1":
                results = graphq1(driver=driver, node_id=disease_id) 
                status_bar.config(text=f"Graph Query 1 executed successfully with disease ID: {disease_id}")
            elif query_selection == "Query 2":
                results = graphq2(driver=driver)  
                status_bar.config(text="Graph Query 2 executed successfully")
        elif db_selection == "Document":
            if query_selection == "Query 1":
                results = mongoq1(node_id=disease_id)  
                status_bar.config(text=f"Mongo Query 1 executed successfully with disease ID: {disease_id}")
            elif query_selection == "Query 2":
                results = mongoq2()  
                status_bar.config(text="Mongo Query 2 executed successfully")
        
        # Display results in the results area and clear previous stuff
        results_text.config(state=tk.NORMAL) 
        results_text.delete(1.0, tk.END)  
        
        if results:
            results_text.insert(tk.END, f"Results from {db_selection} DB {query_selection}:\n\n")
            for i, item in enumerate(results, 1):
                results_text.insert(tk.END, f"{i}. {item}\n")
        else:
            results_text.insert(tk.END, "No results found.")
    
    except Exception as e:
        # Handle any errors
        results_text.config(state=tk.NORMAL)
        results_text.delete(1.0, tk.END)
        results_text.insert(tk.END, f"Error executing query: {str(e)}")
        status_bar.config(text="Error occurred during query execution")
    
    finally:
        results_text.config(state=tk.NORMAL)  
        execute_button.config(state=tk.NORMAL)
        status_bar.config(text="Ready")

# Create the main window
root = tk.Tk()
root.title("Database Query Interface")
root.geometry("600x500")  

style = ttk.Style()
style.configure("TFrame", background="#f0f0f0")
style.configure("TRadiobutton", background="#f0f0f0", font=("Arial", 10))
style.configure("TLabel", background="#f0f0f0", font=("Arial", 10))
style.configure("TButton", font=("Arial", 10, "bold"))

# Main container
main_frame = ttk.Frame(root, padding="20")
main_frame.pack(fill=tk.BOTH, expand=True)

# Database selection frame
db_frame = ttk.LabelFrame(main_frame, text="Select Database", padding="10")
db_frame.pack(fill=tk.X, pady=10)

db_var = tk.StringVar(value="Graph")
graph_rb = ttk.Radiobutton(db_frame, text="Graph Database", variable=db_var, value="Graph")
document_rb = ttk.Radiobutton(db_frame, text="Document Database", variable=db_var, value="Document")
graph_rb.grid(row=0, column=0, padx=20, pady=5, sticky=tk.W)
document_rb.grid(row=0, column=1, padx=20, pady=5, sticky=tk.W)

# Query selection frame
query_frame = ttk.LabelFrame(main_frame, text="Select Query", padding="10")
query_frame.pack(fill=tk.X, pady=10)

query_var = tk.StringVar(value="Query 1")
query1_rb = ttk.Radiobutton(query_frame, text="Query 1 (Requires Disease ID)", variable=query_var, value="Query 1")
query2_rb = ttk.Radiobutton(query_frame, text="Query 2", variable=query_var, value="Query 2")
query1_rb.grid(row=0, column=0, padx=20, pady=5, sticky=tk.W)
query2_rb.grid(row=0, column=1, padx=20, pady=5, sticky=tk.W)

# Disease ID input frame - now always visible
disease_frame = ttk.LabelFrame(main_frame, text="Disease ID Input", padding="10")
disease_frame.pack(fill=tk.X, pady=10, padx=20)

disease_label = ttk.Label(disease_frame, text="Enter Disease ID:")
disease_label.pack(side=tk.LEFT, padx=(0, 10))

disease_entry = ttk.Entry(disease_frame, width=30)
disease_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)

# Help button
help_button = ttk.Button(disease_frame, text="?", width=2, 
                         command=lambda: messagebox.showinfo("Help", "Enter a valid disease ID (e.g., Disease::DOID:986)"))
help_button.pack(side=tk.RIGHT, padx=(5, 0))

# Execute button
button_frame = ttk.Frame(main_frame)
button_frame.pack(pady=10, fill=tk.X)

execute_button = ttk.Button(button_frame, text="Execute Query", command=on_request_button_click)
execute_button.pack(side=tk.TOP, pady=5)

# Results area
results_frame = ttk.LabelFrame(main_frame, text="Results", padding="10")
results_frame.pack(fill=tk.BOTH, expand=True, pady=10)

results_text = scrolledtext.ScrolledText(results_frame, wrap=tk.WORD, width=60, height=10)
results_text.pack(fill=tk.BOTH, expand=True)
results_text.config(state=tk.DISABLED)  # Make read-only initially

# Status bar
status_bar = ttk.Label(root, text="Ready", relief=tk.SUNKEN, anchor=tk.W)
status_bar.pack(side=tk.BOTTOM, fill=tk.X)


# Status update to remind user of disease ID requirement
def update_status_based_on_query(*args):
    if query_var.get() == "Query 1":
        status_bar.config(text="Ready - Disease ID required for Query 1")
    else:
        status_bar.config(text="Ready")

# Add trace for query selection to update status bar
query_var.trace_add("write", update_status_based_on_query)
# Initial status update
update_status_based_on_query()

# Start the GUI
root.mainloop()